In [1]:
import numpy as np
import torch
from tqdm import tqdm
import os

In [9]:
'''
choose between train and val
'''
filename = 'val'
# filename = 'train'

In [10]:
block_size = 256
p = 7
N = 1000000
random_list = np.random.randint(low = 1,high =8, size = (N,block_size//2), dtype = np.int32)
partial_sum_random_list = ( np.cumsum(random_list, axis = 1) -1) % p + 1
input_length_list = np.random.randint(low = 1,high =block_size//2, size = (N,), dtype = np.int8)
token_id_list = []
for i in range(N):
    current_input_tokens = random_list[i][:input_length_list[i]]
    current_label_tokens = partial_sum_random_list[i][:input_length_list[i]]
    token_id_list.append(list(current_input_tokens) + [-2] + list(current_label_tokens))

In [7]:
token_id_list[0]

[4,
 7,
 6,
 2,
 1,
 5,
 6,
 4,
 5,
 5,
 6,
 5,
 2,
 1,
 7,
 2,
 1,
 4,
 1,
 3,
 7,
 4,
 1,
 2,
 4,
 6,
 6,
 2,
 7,
 6,
 7,
 -2,
 4,
 4,
 3,
 5,
 6,
 4,
 3,
 7,
 5,
 3,
 2,
 7,
 2,
 3,
 3,
 5,
 6,
 3,
 4,
 7,
 7,
 4,
 5,
 7,
 4,
 3,
 2,
 4,
 4,
 3,
 3]

In [11]:
dtype = np.int8
arr = np.memmap(filename + '.bin', dtype=dtype, mode='write', shape=(block_size * len(token_id_list)))
arr_label = np.memmap(filename + '_label.bin', dtype=dtype, mode='write', shape=(block_size * len(token_id_list)))
print(f"writing {filename}...")
for jj, example in tqdm(enumerate(token_id_list)):
    arr[jj * block_size : jj * block_size + len(example)] = example
    arr[jj * block_size + len(example) : (jj + 1) * block_size] = 0
    arr_label[jj * block_size : jj * block_size + input_length_list[jj]] = -1
    arr_label[jj * block_size + input_length_list[jj]  : jj * block_size + len(example) - 1] = example[input_length_list[jj] +1 :]
    arr_label[jj * block_size + len(example) - 1 : (jj + 1) * block_size] = -1
    # pad to bock_size
arr.flush()
arr_label.flush()

writing val...


1000000it [00:13, 76484.44it/s]


In [51]:
arr[:100]

memmap([ 6, -2,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  6,  3,  5,  1,  5, -2,  1,  7,  3,  1,  2,  7,  0,  0,  0,
         7,  5,  5,  3, -2,  7,  5,  3,  6,  0,  0,  0,  0,  0,  0,  0,
         2,  3, -2,  2,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         5,  1,  5,  2,  1,  6,  7, -2,  5,  6,  4,  6,  7,  6,  6,  0,
         7, -2,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         4, -2,  4,  0], dtype=int8)

In [5]:
data_dir = os.path.join('data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
train_label = np.memmap(os.path.join(data_dir, 'train_label.bin'), dtype=np.int16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
val_label = np.memmap(os.path.join(data_dir, 'val_label.bin'), dtype=np.int16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    label = train_label if split == 'train' else val_label
    ix = torch.randint(len(data) // block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((label[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

In [40]:
import torch.nn.functional as F
input = torch.randn(10, 10000, requires_grad=True)*100
target = torch.randint(10000, (10,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
print(loss)

tensor(408.9389, grad_fn=<NllLossBackward0>)
